Wanted Crawling(이상현) _ 3팀 PFlow Project(No.3)


Role

- Wanted 데이터 수집
- Wanted 웹크롤링, 데이터 프레임 형성, 파일 생성, SQL 적재 

# 셀레니움 설치
!pip install selenium
# 패키지 설치
!pip install webdriver_manager
# 스텔스 기능 
!pip install selenium-stealth
# 시간 체크 기능
!pip install tqdm

In [ ]:
# 일반적인 크롤링
# Chrome 웹 드라이버 다운로드 및 설치 후 실행
# 설치한 selenium에서 webdriver를 import
# 크롤링 구문에서 사용할 필요 라이브러리 import
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium_stealth import stealth # 스텔스 기능
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm  # 상태바 모듈 불러오기
import random  # random 모듈
import time

In [ ]:
# selenum의 webdriver에 앞서 설치한 chromedirver를 연동
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:
# 접속 사이트 주소 인식
# `driver.get()` 내에 URL 주소를 입력하면 해당 주소로 이동
url = 'https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=job.recommend_order&years=-1&locations=all'
driver.get(url)

In [ ]:
# 스크롤을 가장 아래로 내리는 함수
def scroll_to_bottom(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 페이지의 가장 아래로 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # 새로운 내용이 로드될 때까지 대기
        time.sleep(2)
        
        # 새로운 페이지 높이 측정
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # 새로운 콘텐츠가 없으면 종료
        if new_height == last_height:
            break
        
        last_height = new_height

# 스크롤 함수 호출
scroll_to_bottom(driver)

In [ ]:
# 크롤링 가능한 페이지 길이 확인
driver.execute_script('return document.body.scrollHeight')

In [ ]:
# BeautifulSoup으로 파싱
from bs4 import BeautifulSoup

req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')

In [ ]:
# 페이지에 몇개 공고문 있는지 확인
content = soup.select('.Card_Card__WdaEk')
len(content)

In [ ]:
# 타이틀 문자열 확인
content[1].get_text()

In [ ]:
# BeautifulSoup으로 파싱
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')

In [ ]:
# 모든 타이틀 불러오기
soup.find_all('p', class_ = 'Typography_Typography__root__RdAI1 Typography_Typography__body2__5Mmhi Typography_Typography__weightBold__KkJEY JobCard_JobCard__body__position__CyaGY')

In [ ]:
# 화면 최상단으로 이동
driver.execute_script('window.scrollTo(0, 0);')

In [ ]:
# url_base = "https://www.wanted.co.kr/"
# url_sub = "wdlist/518?country=kr&job_sort=job.recommend_order&years=-1&selected=1025&selected=1024&selected=655&selected=1634&selected=669&locations=all"
# url = url_base + url_sub
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')
soup.prettify()

In [ ]:
# 공고문 제목만 불러오기
title = soup.find_all("p", class_ = "Typography_Typography__root__RdAI1 Typography_Typography__body2__5Mmhi Typography_Typography__weightBold__KkJEY JobCard_JobCard__body__position__CyaGY")
title

In [ ]:
# 타이틀 제목 리스트로 형변환 하고 몇개 데이터 있는지 길이 확인
title_list = [item.text for item in title]
title_list, len(title_list)

In [ ]:
# 회사 이름 확인
company = soup.find_all("span", class_ = "Typography_Typography__root__RdAI1 Typography_Typography__label2__svmAA Typography_Typography__weightMedium__GXnOM JobCard_JobCard__body__company__AUj_B")
company

In [ ]:
# 포문으로 회사 이름 리스트로 생성, 요소 갯수 확인
company_list = [item.text for item in company]
company_list, len(company_list)

In [ ]:
# 회사 이름, 공고문 데이터 프레임 생성
import pandas as pd

In [ ]:
df = pd.DataFrame(company_list)
df

In [ ]:
df.rename(columns = {df.columns[0] : '회사명'}, inplace= True)
df

In [ ]:
df['제목'] = (title_list)
df.head()

In [ ]:
from urllib.parse import urljoin

In [ ]:
url_base = "https://www.wanted.co.kr/"
url_sub = "wdlist/518/1634?country=kr&job_sort=job.recommend_order&years=-1&selected=1634&locations=seoul.gangnam-gu&locations=seoul.guro-gu&locations=seoul.geumcheon-gu&locations=seoul.seocho-gu&locations=seoul.songpa-gu"
url = url_base + url_sub

In [ ]:
soup_list = soup.find_all('div', 'JobCard_JobCard__Tb7pI')
soup_list

In [ ]:
# for문으로  url 데이터 리스트로 추출
url_add = []
for item in soup_list:
    url_add.append(urljoin(url_base, item.find('a')['href']))

In [ ]:
url_add

In [ ]:
# 데이터 프레임에 url 컬럼 추가
df['URL'] = url_add
df

페이지 내부 파싱 및 크롤링(비활성화)

from urllib.request import Request, urlopen

req =Request(df["URL"][0], headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36'})
html = urlopen(req)
soup_tmp = BeautifulSoup(html, 'html.parser')
soup_tmp.find('div', class_ = 'JobHeader_JobHeader__Tools__80TiC')

driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div[2]/ul/li[1]/div/a/div[2]/p').click()

element_detail = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button/span[2]').click()

element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/div').text
element.split('∙')

driver.back()

df

데이터프레임 내 웹페이지 주소 기반 크롤링 진행

## 1. 저속 기본 크롤링 셀레니움(240825)


# 크롬 드라이버 설정
driver = webdriver.Chrome()

# 크롤링 데이터 저장을 위한 리스트
title_list = []         # 제목 리스트
company_list = []       # 회사 이름 리스트
career_list = []        # 경력 리스트
mainjob_list = []       # 주요업무 리스트
qual_list = []          # 자격요건 리스트
prefer_list = []        # 우대사항 리스트
welfare_list = []       # 혜택 및 복지 리스트
deadline_list = []      # 마감일 리스트
location_list = []      # 근무지 리스트

# 데이터프레임의 각 행에 대해 반복
for idx, row in df.iterrows():
    url = row['URL']
    try:
        # URL 접속
        driver.get(url)
        time.sleep(3)  # 페이지가 로드될 시간을 충분히 기다림

        try:
            # 페이지 스크롤을 조금씩 내리면서 요소를 찾음
            while True:
                driver.execute_script("window.scrollBy(0, 500);")  # 500픽셀씩 스크롤
                time.sleep(1)  # 스크롤 후 잠시 대기

                try:
                    # 상세보기 버튼 찾기
                    detail_button_xpath = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f'//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button/span[1]'))
                    )
                    driver.execute_script("arguments[0].click();", detail_button_xpath)
                    break  # 클릭이 성공하면 루프를 종료
                except (TimeoutException, NoSuchElementException):
                    pass  # 아직 버튼이 보이지 않으면 다시 시도
        except TimeoutException:
            print("상세보기 버튼을 찾을 수 없습니다.")
            
        time.sleep(3)
        
        # html 코드 파싱
        soup_tmp = BeautifulSoup(driver.page_source, 'html.parser')
        
        # 페이지 제목 가져오기
        try:        
            title = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/h1').text
        except NoSuchElementException:
            title = "제목 없음"
        time.sleep(2)

        # 회사명 가져오기
        try:        
            company = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/div/div[1]/a').text
        except NoSuchElementException:
            company = "회사명 없음"
        time.sleep(2)

        # 경력 가져오기
        try:        
            career = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/div/div[1]/span[4]').text
        except NoSuchElementException:
            career = "경력 없음"
        time.sleep(2)

        # 주요업무 가져오기
        try:
            mainjob = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/div[1]').text
        except NoSuchElementException:
            mainjob = "주요 업무 없음"
        time.sleep(2)    

        # 자격요건 가져오기
        try:
            qual = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/div[2]').text
        except NoSuchElementException:
            qual = "자격 요건 없음"
        time.sleep(2)    

        # 우대사항 가져오기
        try:
            prefer = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/div[3]').text
        except NoSuchElementException:
            prefer = "우대 사항 없음"
        time.sleep(2)

        # 복지 가져오기
        try:
            welfare = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/div[4]').text
        except NoSuchElementException:
            welfare = "복지 정보 없음"
        time.sleep(2)

        # 마감일 가져오기
        try:        
            deadline = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[4]/span').text
        except NoSuchElementException:
            deadline = "마감 정보 없음"
        time.sleep(2)

        # 근무지 가져오기
        try:        
            location = location = soup_tmp.find('span', class_='Typography_Typography__root__RdAI1 Typography_Typography__body2__5Mmhi Typography_Typography__weightMedium__GXnOM').get_text()
        except NoSuchElementException:
            location = "근무지 정보 없음"

        # 해당 정보 빈 리스트에 저장
        title_list.append(title)
        company_list.append(company)
        career_list.append(career)
        mainjob_list.append(mainjob)
        qual_list.append(qual)
        prefer_list.append(prefer)
        welfare_list.append(welfare)
        deadline_list.append(deadline)
        location_list.append(location)
        
    except Exception as e:
        print(f"페이지를 로드할 수 없거나, 예상치 못한 오류 발생: {e}")
        # 예외 발생 시 각 리스트에 예외 처리 메시지 추가
        title_list.append("페이지 만료")
        company_list.append("페이지 만료")
        career_list.append("페이지 만료")
        mainjob_list.append("페이지 만료")
        qual_list.append("페이지 만료")
        prefer_list.append("페이지 만료")
        welfare_list.append("페이지 만료")
        deadline_list.append("페이지 만료")
        location_list.append("페이지 만료")
        
    # 진행도 확인하기 위해 idx 출력
    print(f"Processed row: {idx}")

## 2. 고속화 크롤링 뷰티풀 숲(240826)


# 크롬 드라이버 설정
driver = webdriver.Chrome()

# 크롤링 데이터 저장을 위한 리스트
title_list = []         # 제목 리스트
company_list = []       # 회사 이름 리스트
career_list = []        # 경력 리스트
mainjob_list = []       # 주요업무 리스트
qual_list = []          # 자격요건 리스트
prefer_list = []        # 우대사항 리스트
welfare_list = []       # 혜택 및 복지 리스트
deadline_list = []      # 마감일 리스트
location_list = []      # 근무지 리스트

# 데이터프레임의 각 행에 대해 반복
for idx, row in df.iterrows():
    url = row['URL']
    try:
        # URL 접속
        driver.get(url)
        
        # 페이지 로드 완료 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/h1'))
        )
        
        # 상세보기 버튼 찾기
        try:
            while True:
                # 스크롤을 통해 상세보기 버튼을 찾음
                driver.execute_script("window.scrollBy(0, 500);")  # 500픽셀씩 스크롤
                try:
                    detail_button = WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button/span[1]'))
                    )
                    driver.execute_script("arguments[0].click();", detail_button)
                    break  # 클릭이 성공하면 루프를 종료
                except (TimeoutException, NoSuchElementException):
                    pass  # 아직 버튼이 보이지 않으면 다시 시도
        except TimeoutException:
            print("상세보기 버튼을 찾을 수 없습니다.")
        
        # 대기 시간 설정
        time.sleep(random.uniform(1,3))
        
        # 페이지 로드 후 소스 파싱
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # 페이지 제목 가져오기
        title = soup.select_one('h1').text if soup.select_one('h1') else "제목 없음"

        # 회사명 가져오기
        company = soup.select_one('header div a').text if soup.select_one('header div a') else "회사명 없음"

        # 경력 가져오기
        career = soup.select_one('header div span:nth-of-type(4)').text if soup.select_one('header div span:nth-of-type(4)') else "경력 없음"

        # 주요업무 가져오기
        mainjob = soup.select_one('article div div:nth-of-type(1)').text if soup.select_one('article div div:nth-of-type(1)') else "주요 업무 없음"

        # 자격요건 가져오기
        qual = soup.select_one('article div div:nth-of-type(2)').text if soup.select_one('article div div:nth-of-type(2)') else "자격 요건 없음"

        # 우대사항 가져오기
        prefer = soup.select_one('article div div:nth-of-type(3)').text if soup.select_one('article div div:nth-of-type(3)') else "우대 사항 없음"

        # 복지 가져오기
        welfare = soup.select_one('article div div:nth-of-type(4)').text if soup.select_one('article div div:nth-of-type(4)') else "복지 정보 없음"

        # 마감일 가져오기
        deadline = soup.select_one('article:nth-of-type(4) span').text if soup.select_one('article:nth-of-type(4) span') else "마감 정보 없음"

        # 근무지 가져오기
        location = soup.find('span', class_='Typography_Typography__root__RdAI1 Typography_Typography__body2__5Mmhi Typography_Typography__weightMedium__GXnOM').text if soup.find('span', class_='Typography_Typography__root__RdAI1 Typography_Typography__body2__5Mmhi Typography_Typography__weightMedium__GXnOM') else "근무지 정보 없음"

        # 해당 정보 빈 리스트에 저장
        title_list.append(title)
        company_list.append(company)
        career_list.append(career)
        mainjob_list.append(mainjob)
        qual_list.append(qual)
        prefer_list.append(prefer)
        welfare_list.append(welfare)
        deadline_list.append(deadline)
        location_list.append(location)
        
    except Exception as e:
        print(f"페이지를 로드할 수 없거나, 예상치 못한 오류 발생: {e}")
        # 예외 발생 시 각 리스트에 예외 처리 메시지 추가
        title_list.append("페이지 만료")
        company_list.append("페이지 만료")
        career_list.append("페이지 만료")
        mainjob_list.append("페이지 만료")
        qual_list.append("페이지 만료")
        prefer_list.append("페이지 만료")
        welfare_list.append("페이지 만료")
        deadline_list.append("페이지 만료")
        location_list.append("페이지 만료")
        
    # 진행도 확인하기 위해 idx 출력
    print(f"Processed row: {idx}")

## 3. 고속화 크롤링 뷰티풀숲, 스텔스 기능 (240827) 81.56초


# 크롬 드라이버 설정
driver = webdriver.Chrome()

# 스텔스 기능 적용
stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
)

# 크롤링 데이터 저장을 위한 리스트
title_list = []         # 제목 리스트
company_list = []       # 회사 이름 리스트
career_list = []        # 경력 리스트
mainjob_list = []       # 주요업무 리스트
qual_list = []          # 자격요건 리스트
prefer_list = []        # 우대사항 리스트
welfare_list = []       # 혜택 및 복지 리스트
deadline_list = []      # 마감일 리스트
location_list = []      # 근무지 리스트

# 크롤링 시작 시간 기록
start_time = time.time()

# 데이터프레임의 각 행에 대해 반복
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Crawling Progress"):
    url = row['URL']
    try:
        # URL 접속
        driver.get(url)
        
        # 페이지 로드 완료 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/h1'))
        )
        
        # 상세보기 버튼 찾기
        try:
            while True:
                # 스크롤을 통해 상세보기 버튼을 찾음
                driver.execute_script("window.scrollBy(0, 500);")  # 500픽셀씩 스크롤
                try:
                    detail_button = WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button/span[1]'))
                    )
                    driver.execute_script("arguments[0].click();", detail_button)
                    break  # 클릭이 성공하면 루프를 종료
                except (TimeoutException, NoSuchElementException):
                    pass  # 아직 버튼이 보이지 않으면 다시 시도
        except TimeoutException:
            print("상세보기 버튼을 찾을 수 없습니다.")
        
        # 대기 시간 설정
        time.sleep(random.uniform(1,3))
        
        # 페이지 로드 후 소스 파싱
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # 페이지 제목 가져오기
        title = soup.select_one('h1').text if soup.select_one('h1') else "제목 없음"

        # 회사명 가져오기
        company = soup.select_one('header div a').text if soup.select_one('header div a') else "회사명 없음"

        # 경력 가져오기
        career = soup.select_one('header div span:nth-of-type(4)').text if soup.select_one('header div span:nth-of-type(4)') else "경력 없음"

        # 주요업무 가져오기
        mainjob = soup.select_one('article div div:nth-of-type(1)').text if soup.select_one('article div div:nth-of-type(1)') else "주요 업무 없음"

        # 자격요건 가져오기
        qual = soup.select_one('article div div:nth-of-type(2)').text if soup.select_one('article div div:nth-of-type(2)') else "자격 요건 없음"

        # 우대사항 가져오기
        prefer = soup.select_one('article div div:nth-of-type(3)').text if soup.select_one('article div div:nth-of-type(3)') else "우대 사항 없음"

        # 복지 가져오기
        welfare = soup.select_one('article div div:nth-of-type(4)').text if soup.select_one('article div div:nth-of-type(4)') else "복지 정보 없음"

        # 마감일 가져오기
        deadline = soup.select_one('article:nth-of-type(4) span').text if soup.select_one('article:nth-of-type(4) span') else "마감 정보 없음"

        # 근무지 가져오기
        location = soup.find('span', class_='Typography_Typography__root__RdAI1 Typography_Typography__body2__5Mmhi Typography_Typography__weightMedium__GXnOM').text if soup.find('span', class_='Typography_Typography__root__RdAI1 Typography_Typography__body2__5Mmhi Typography_Typography__weightMedium__GXnOM') else "근무지 정보 없음"

        # 해당 정보 빈 리스트에 저장
        title_list.append(title)
        company_list.append(company)
        career_list.append(career)
        mainjob_list.append(mainjob)
        qual_list.append(qual)
        prefer_list.append(prefer)
        welfare_list.append(welfare)
        deadline_list.append(deadline)
        location_list.append(location)
        
    except Exception as e:
        print(f"페이지를 로드할 수 없거나, 예상치 못한 오류 발생: {e}")
        # 예외 발생 시 각 리스트에 예외 처리 메시지 추가
        title_list.append("페이지 만료")
        company_list.append("페이지 만료")
        career_list.append("페이지 만료")
        mainjob_list.append("페이지 만료")
        qual_list.append("페이지 만료")
        prefer_list.append("페이지 만료")
        welfare_list.append("페이지 만료")
        deadline_list.append("페이지 만료")
        location_list.append("페이지 만료")
        
    # 진행도 확인하기 위해 idx 출력
    print(f"Processed row: {idx}")
    
# 크롤링 종료 시간 기록
end_time = time.time()

# 크롤링 총 시간 계산 및 출력
total_time = end_time - start_time
print(f"크롤링 작업 완료. 총 소요 시간: {total_time:.2f}초")

In [ ]:
## 4. 고속화 크롤링 뷰티풀숲, 스텔스, 헤드리스, 이미지로드 비활성화, user-Agent변경 기능 적용 (240827) 79.27초 소요
## 완성 1 형

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium_stealth import stealth

# 크롬 옵션 설정
chrome_options = Options()

# 헤드리스 모드
chrome_options.add_argument("--headless")  
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 이미지 로드 비활성화
chrome_options.add_argument("--blink-settings=imagesEnabled=false")

# User-Agent 변경
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36")

# 크롬 드라이버 설정
driver = webdriver.Chrome(options=chrome_options)

# 스텔스 기능 적용
stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
)

# 크롤링 데이터 저장을 위한 리스트
title_list = []         # 제목 리스트
company_list = []       # 회사 이름 리스트
career_list = []        # 경력 리스트
mainjob_list = []       # 주요업무 리스트
qual_list = []          # 자격요건 리스트
prefer_list = []        # 우대사항 리스트
welfare_list = []       # 혜택 및 복지 리스트
skill_list = []         # 스킬 리스트
tag_list = []           # 태그 리스트
deadline_list = []      # 마감일 리스트
location_list = []      # 근무지 리스트

# 크롤링 시작 시간 기록
start_time = time.time()

# 데이터프레임의 각 행에 대해 반복
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Crawling Progress", ncols=100, position=0, leave=True):
    url = row['URL']
    try:
        # URL 접속
        driver.get(url)
        
        # 페이지 로드 완료 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/h1'))
        )
        
        # 상세보기 버튼 찾기
        try:
            while True:
                # 스크롤을 통해 상세보기 버튼을 찾음
                driver.execute_script("window.scrollBy(0, 500);")  # 500픽셀씩 스크롤
                try:
                    detail_button = WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button/span[1]'))
                    )
                    driver.execute_script("arguments[0].click();", detail_button)
                    break  # 클릭이 성공하면 루프를 종료
                except (TimeoutException, NoSuchElementException):
                    pass  # 아직 버튼이 보이지 않으면 다시 시도
        except TimeoutException:
            print("상세보기 버튼을 찾을 수 없습니다.")
        
        # 대기 시간 설정
        time.sleep(random.uniform(1,3))
        
        # 페이지 로드 후 소스 파싱
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # 페이지 제목 가져오기
        title = soup.select_one('h1').text if soup.select_one('h1') else "제목 없음"

        # 회사명 가져오기
        company = soup.select_one('header div a').text if soup.select_one('header div a') else "회사명 없음"

        # 경력 가져오기
        career = soup.select_one('header div span:nth-of-type(4)').text if soup.select_one('header div span:nth-of-type(4)') else "경력 없음"

        # 주요업무 가져오기
        mainjob = soup.select_one('article div div:nth-of-type(1)').text if soup.select_one('article div div:nth-of-type(1)') else "주요 업무 없음"

        # 자격요건 가져오기
        qual = soup.select_one('article div div:nth-of-type(2)').text if soup.select_one('article div div:nth-of-type(2)') else "자격 요건 없음"

        # 우대사항 가져오기
        prefer = soup.select_one('article div div:nth-of-type(3)').text if soup.select_one('article div div:nth-of-type(3)') else "우대 사항 없음"

        # 복지 가져오기
        welfare = soup.select_one('article div div:nth-of-type(4)').text if soup.select_one('article div div:nth-of-type(4)') else "복지 정보 없음"

        # 스킬 가져오기
        skill = soup.select_one('.JobSkillTags_JobSkillTags__list__01GRk').text if soup.select_one('.JobSkillTags_JobSkillTags__list__01GRk') else "스킬 없음"

        # 태그 가져오기
        tag = soup.select_one('.CompanyTags_CompanyTags__list__WjcTV').text if soup.select_one('.CompanyTags_CompanyTags__list__WjcTV') else "태그 없음"
        
        # 마감일 가져오기
        deadline = soup.select_one('.JobDueTime_JobDueTime__3yzxa').text if soup.select_one('.JobDueTime_JobDueTime__3yzxa') else "마감 정보 없음"

        # 근무지 가져오기
        location = soup.find('span', class_='Typography_Typography__root__RdAI1 Typography_Typography__body2__5Mmhi Typography_Typography__weightMedium__GXnOM').text if soup.find('span', class_='Typography_Typography__root__RdAI1 Typography_Typography__body2__5Mmhi Typography_Typography__weightMedium__GXnOM') else "근무지 정보 없음"

        # 해당 정보 빈 리스트에 저장
        title_list.append(title)
        company_list.append(company)
        career_list.append(career)
        mainjob_list.append(mainjob)
        qual_list.append(qual)
        prefer_list.append(prefer)
        welfare_list.append(welfare)
        skill_list.append(skill)
        tag_list.append(tag)
        deadline_list.append(deadline)
        location_list.append(location)
        
    except Exception as e:
        print(f"페이지를 로드할 수 없거나, 예상치 못한 오류 발생: {e}")
        # 예외 발생 시 각 리스트에 예외 처리 메시지 추가
        title_list.append("페이지 만료")
        company_list.append("페이지 만료")
        career_list.append("페이지 만료")
        mainjob_list.append("페이지 만료")
        qual_list.append("페이지 만료")
        prefer_list.append("페이지 만료")
        welfare_list.append("페이지 만료")
        skill_list.append("페이지 만료")
        tag_list.append("페이지 만료")
        deadline_list.append("페이지 만료")
        location_list.append("페이지 만료")
        
# 드라이버 종료
driver.quit()
    
# 크롤링 종료 시간 기록
end_time = time.time()

# 크롤링 총 시간 계산 및 출력
total_time = end_time - start_time
print(f"크롤링 작업 완료. 총 소요 시간: {total_time:.2f}초")

## 5. 고속화 크롤링 뷰티풀숲, 스텔스, 헤드리스, 이미지로드 비활성화, user-Agent변경, 병렬작업 기능 적용 (240827) 467초 소요(오류 발생)
## 완성 2형

# 크롬 옵션 설정
chrome_options = Options()

# 헤드리스 모드 및 기타 옵션 설정
chrome_options.add_argument("--headless")  
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36")

# 크롤링 데이터 저장을 위한 리스트
title_list = []         # 제목 리스트
company_list = []       # 회사 이름 리스트
career_list = []        # 경력 리스트
mainjob_list = []       # 주요업무 리스트
qual_list = []          # 자격요건 리스트
prefer_list = []        # 우대사항 리스트
welfare_list = []       # 혜택 및 복지 리스트
skill_list = []         # 스킬 리스트
tag_list = []           # 태그 리스트
deadline_list = []      # 마감일 리스트
location_list = []      # 근무지 리스트

# URL 크롤링 함수 정의
def crawl_data(url):
    try:
        # 크롬 드라이버 설정
        driver = webdriver.Chrome(options=chrome_options)
        
        # 스텔스 기능 적용
        stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
        )

        # URL 접속
        driver.get(url)

        # 페이지 로드 완료 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/h1'))
        )

        # 상세보기 버튼 찾기 및 클릭
        while True:
            driver.execute_script("window.scrollBy(0, 500);")
            try:
                detail_button = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button/span[1]'))
                )
                driver.execute_script("arguments[0].click();", detail_button)
                break
            except (TimeoutException, NoSuchElementException):
                pass

        time.sleep(random.uniform(1, 3))

        # 페이지 로드 후 소스 파싱
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 크롤링할 데이터 추출
        title = soup.select_one('h1').text if soup.select_one('h1') else "제목 없음"
        company = soup.select_one('header div a').text if soup.select_one('header div a') else "회사명 없음"
        career = soup.select_one('header div span:nth-of-type(4)').text if soup.select_one('header div span:nth-of-type(4)') else "경력 없음"
        mainjob = soup.select_one('article div div:nth-of-type(1)').text if soup.select_one('article div div:nth-of-type(1)') else "주요 업무 없음"
        qual = soup.select_one('article div div:nth-of-type(2)').text if soup.select_one('article div div:nth-of-type(2)') else "자격 요건 없음"
        prefer = soup.select_one('article div div:nth-of-type(3)').text if soup.select_one('article div div:nth-of-type(3)') else "우대 사항 없음"
        welfare = soup.select_one('article div div:nth-of-type(4)').text if soup.select_one('article div div:nth-of-type(4)') else "복지 정보 없음"
        skill = soup.select_one('.JobSkillTags_JobSkillTags__list__01GRk').text if soup.select_one('.JobSkillTags_JobSkillTags__list__01GRk') else "스킬 없음"
        tag = soup.select_one('.CompanyTags_CompanyTags__list__WjcTV').text if soup.select_one('.CompanyTags_CompanyTags__list__WjcTV') else "태그 없음"
        deadline = soup.select_one('.JobDueTime_JobDueTime__3yzxa').text if soup.select_one('.JobDueTime_JobDueTime__3yzxa') else "마감 정보 없음"
        location = soup.find('span', class_='Typography_Typography__root__RdAI1 Typography_Typography__body2__5Mmhi Typography_Typography__weightMedium__GXnOM').text if soup.find('span', class_='Typography_Typography__root__RdAI1 Typography_Typography__body2__5Mmhi Typography_Typography__weightMedium__GXnOM') else "근무지 정보 없음"

        driver.quit()

        return {
            "title": title,
            "company": company,
            "career": career,
            "mainjob": mainjob,
            "qual": qual,
            "prefer": prefer,
            "welfare": welfare,
            "skill": skill,
            "tag": tag,
            "deadline": deadline,
            "location": location
        }
    
    except Exception as e:
        print(f"페이지를 로드할 수 없거나, 예상치 못한 오류 발생: {e}")
        return {
            "title": "페이지 만료",
            "company": "페이지 만료",
            "career": "페이지 만료",
            "mainjob": "페이지 만료",
            "qual": "페이지 만료",
            "prefer": "페이지 만료",
            "welfare": "페이지 만료",
            "skill": "페이지 만료",
            "tag": "페이지 만료",
            "deadline": "페이지 만료",
            "location": "페이지 만료"
        }

# 크롤링 시작 시간 기록
start_time = time.time()

# 병렬 크롤링 실행
with ThreadPoolExecutor(max_workers=4) as executor:  # 2개의 스레드 사용
    futures = [executor.submit(crawl_data, row['URL']) for idx, row in df.iterrows()]
    
    for future in tqdm(as_completed(futures), total=len(futures), desc="Crawling Progress"):
        result = future.result()
        title_list.append(result["title"])
        company_list.append(result["company"])
        career_list.append(result["career"])
        mainjob_list.append(result["mainjob"])
        qual_list.append(result["qual"])
        prefer_list.append(result["prefer"])
        welfare_list.append(result["welfare"])
        skill_list.append(result["skill"])
        tag_list.append(result["tag"])
        deadline_list.append(result["deadline"])
        location_list.append(result["location"])

# 크롤링 종료 시간 기록
end_time = time.time()

# 크롤링 총 시간 계산 및 출력
total_time = end_time - start_time
print(f"크롤링 작업 완료. 총 소요 시간: {total_time:.2f}초")

데이터 프레임 정리 후 파일 저장

In [ ]:
# pymysql 설치(DB적재)
!pip install pymysql

In [ ]:
# 데이터 프레임 형성 및 적재 목적 라이브러리 import
import pandas as pd
import pymysql
import numpy as np

In [ ]:
# 데이터 프레임에 데이터 추가
df['Title'] = title_list
df['Company'] = company_list
df['Career'] = career_list
df['Mainjob'] = mainjob_list
df['Qualified'] = qual_list
df['Preference'] = prefer_list
df['Welfare'] = welfare_list
df['Skill'] = skill_list
df['Tag'] = tag_list
df['Deadline'] = deadline_list
df['Location'] = location_list
df.head()

In [ ]:
df = df.loc[:, ['Company', 'Title', 'Career', 'Mainjob', 'Qualified', 'Preference', 'Welfare', 'Skill', 'Tag', 'Deadline', 'Location', 'URL']]
df.head()

In [ ]:
# 데이터 csv파일로 저장
df.to_csv('data/Wanted_crawling_data.csv', sep=",", encoding='cp949', errors='replace')

# 데이터프레임 형성 및 구를 기준으로 색인하기 위한 인덱스 설정(데이터 전처리 전 기준점에 맞춰 컬럼 설정)
data = {'Title':title_list, 'Company':company_list, 'Career':career_list, 'Mainjob':mainjob_list, 'Qualified':qual_list, '':edu_list, 'InfoDetails':personality_list, 'Tag':em_type_list, 'D_day':pay_list}
df = pd.DataFrame(data)
df.set_index('Address', inplace=True)
df

전처리 작업 후 데이터 적재

In [ ]:
# 주소를 구만 남김
address_list = [item[-3:] for item in address_list]